In [1]:
from mlapp import GitHubApp
from IPython import display

In [2]:
app_id = 27079
key_file_path = '/Users/hamelsmu/.ssh/hamel-python-app.2019-03-15.private-key.pem'

## Get the app installation

The test installation automatically fetches the first installation the app is found on.  This app is only installed in one place, so this just fetches the installation made on the public `hamelsmu/example-github-app` repo. 

In [3]:
ghapp = GitHubApp(pem_path=key_file_path, app_id=app_id)
install = ghapp.get_test_installation()

## Interact With Issues

#### Create a new issue

In [4]:
issue = install.create_issue(owner='hamelsmu',
                             repository='example-github-app',
                             title='test issue', 
                             body='this is automatically created')

#### Comment on an issue

In [5]:
issue.create_comment('Wooo!  Its time to do some machine learning!')

<IssueComment [ml-auto-labeler[bot]]>

#### Add a label to an issue

In [6]:
issue.add_labels('AI-is-taking-over')

[<ShortLabel [AI-is-taking-over]>]

#### See the issue here

In [7]:
print(issue.html_url)

display.Markdown(f'[{issue.html_url}]({issue.html_url})')

https://github.com/hamelsmu/example-github-app/issues/9


[https://github.com/hamelsmu/example-github-app/issues/9](https://github.com/hamelsmu/example-github-app/issues/9)

## Extract Data For Training ML Models

`Issue, Label` pairs

Get a list of issues which you can use to train models

In [8]:
issues = GitHubApp.unpack_issues(client=install, 
                                 owner='kubeflow',
                                 repo='kubeflow')

100%|██████████| 486/486 [03:25<00:00,  3.05it/s]


In [9]:
print(f'there are {len(issues)} issues with labels')

there are 371 issues with labels


In [10]:
print(issues[0].title)
print('\n==Issue Body==')
print(issues[0].body)
print('\n==Labels==')
print(issues[0].labels)
print('\n==Labels==')
print(issues[0].url)

add spartakus as component

==Issue Body==
- Adds spartakus as default component.
- Adds a new flag `disable_usage_report` to `kfctl init`.  If the flag is set, will not create spartakus as component.
- Adds warning message in `coordinator.Generate` if spartakus is found in components list.

<!-- Reviewable:start -->
---
This change is [<img src="https://reviewable.io/review_button.svg" height="34" align="absmiddle" alt="Reviewable"/>](https://reviewable.io/reviews/kubeflow/kubeflow/2784)
<!-- Reviewable:end -->


==Labels==
['cla: yes', 'size/M']

==Labels==
https://github.com/kubeflow/kubeflow/pull/2784


# Build Model

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from textacy.preprocess import preprocess_text
from sklearn.ensemble import RandomForestClassifier

def textacy_cleaner(text: str) -> str:
    """
    Defines the default function for cleaning text.
    This function operates over a list.
    """
    return preprocess_text(text,
                           fix_unicode=True,
                           lowercase=True,
                           transliterate=True,
                           no_urls=True,
                           no_emails=True,
                           no_phone_numbers=True,
                           no_numbers=True,
                           no_currency_symbols=True,
                           no_punct=True,
                           no_contractions=False,
                           no_accents=True)

#### Get Raw data and labels

In [231]:
raw_text = []

for issue in issues:
    raw_text.append(textacy_cleaner(issue.title) + ' ' + textacy_cleaner(issue.body))

In [232]:
labels = [issue.labels for issue in issues]

assert len(raw_text) == len(raw_text)

#### See top labels in Kubeflow

In [233]:
from collections import Counter
import numpy as np
c = Counter()

for label in labels:
    c.update(label)
    
# only model on labels that occur at least 10 times
labels_to_keep = [x[0] for x in c.most_common() if x[1]>=10]

labels_to_keep

['priority/p2',
 'priority/p1',
 'area/jupyter',
 'community/question',
 'platform/gcp',
 'area/kfctl',
 'area/testing',
 'cla: yes',
 'area/inference',
 'area/bootstrap',
 'area/0.4.0',
 'help wanted',
 'cuj/multi-user',
 'area/build-release',
 'area/0.5.0',
 'cuj/build-train-deploy',
 'area/front-end']

In [234]:
labels_to_keep = ['area/jupyter',
 'community/question',
 'platform/gcp',
 'area/kfctl',
 'area/testing',
 'cla: yes',
 'area/inference',
 'area/bootstrap',
 'help wanted',
 'cuj/multi-user',
 'area/build-release',
 'cuj/build-train-deploy',
 'area/front-end']

In [235]:
labels = [[x for x in x if x in labels_to_keep] for x in labels]

#### Filter out records that do not contain one of the top labels in the dataset

In [236]:
mask = [bool(x) for x in labels]

In [237]:
print(f'There are {sum(mask)} issues out of the original {len(issues)} that have one of the top 10 labels')

There are 293 issues out of the original 371 that have one of the top 10 labels


In [238]:
filtered_text = np.array(raw_text)[mask]
filtered_labels = np.array(labels)[mask]

In [239]:
assert len(filtered_text) == sum(mask) == len(filtered_labels)

#### Train / Test Split

In [240]:
text_train, text_val, labels_train, labels_val = train_test_split(filtered_text, filtered_labels, test_size=.15)

In [241]:
assert len(text_train) == len(labels_train)
assert len(text_val) == len(labels_val)

#### Apply TFIDF Transformation to data

In [242]:
tfidf = TfidfVectorizer(min_df=3)

Xtrain = tfidf.fit_transform(text_train)
Xtest = tfidf.transform(text_val)

In [243]:
mlb = MultiLabelBinarizer()

ytrain = mlb.fit_transform(labels_train)
ytest = mlb.transform(labels_val)

In [244]:
assert ytrain.shape[0] == Xtrain.shape[0]
assert ytest.shape[0] == Xtest.shape[0]

#### Train Model

In [308]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

In [246]:
input_dim = Xtrain.shape[1]
output_dim = ytrain.shape[1]

In [247]:
input_dim

979

In [248]:
output_dim

13

In [349]:
inp = Input(shape=(input_dim,))
h1 = Dense(100)(inp)
out = Dense(output_dim, activation='sigmoid')(h1)
mc = ModelCheckpoint(filepath='best_model.hdf5', save_best_only=True)

adam = optimizers.Adam(lr = .01)

model = Model(inp, out)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [350]:
model.fit(Xtrain, ytrain, validation_data=[Xtest, ytest], epochs=100, callbacks=[mc])

Train on 249 samples, validate on 44 samples
Epoch 1/100
249/249 [==============================] - 1s 6ms/sample - loss: 3.2893 - acc: 0.3092 - val_loss: 3.0053 - val_acc: 0.3864
Epoch 2/100
249/249 [==============================] - 0s 185us/sample - loss: 2.7047 - acc: 0.6145 - val_loss: 2.7907 - val_acc: 0.3182
Epoch 3/100
249/249 [==============================] - 0s 209us/sample - loss: 1.8893 - acc: 0.7189 - val_loss: 2.3120 - val_acc: 0.3409
Epoch 4/100
249/249 [==============================] - 0s 205us/sample - loss: 1.0408 - acc: 0.8554 - val_loss: 1.9911 - val_acc: 0.4545
Epoch 5/100
249/249 [==============================] - 0s 146us/sample - loss: 0.6956 - acc: 0.8594 - val_loss: 2.1737 - val_acc: 0.4545
Epoch 6/100
249/249 [==============================] - 0s 143us/sample - loss: 0.6087 - acc: 0.8474 - val_loss: 2.0862 - val_acc: 0.4773
Epoch 7/100
249/249 [==============================] - 0s 143us/sample - loss: 0.5968 - acc: 0.8715 - val_loss: 2.3376 - val_acc: 0.431

249/249 [==============================] - 0s 158us/sample - loss: 0.4935 - acc: 0.8675 - val_loss: 2.6948 - val_acc: 0.5000
Epoch 61/100
249/249 [==============================] - 0s 159us/sample - loss: 0.4933 - acc: 0.7992 - val_loss: 2.7063 - val_acc: 0.5000
Epoch 62/100
249/249 [==============================] - 0s 150us/sample - loss: 0.4933 - acc: 0.8514 - val_loss: 2.7105 - val_acc: 0.5000
Epoch 63/100
249/249 [==============================] - 0s 149us/sample - loss: 0.4932 - acc: 0.8193 - val_loss: 2.7279 - val_acc: 0.5000
Epoch 64/100
249/249 [==============================] - 0s 165us/sample - loss: 0.4932 - acc: 0.8916 - val_loss: 2.7324 - val_acc: 0.5000
Epoch 65/100
249/249 [==============================] - 0s 184us/sample - loss: 0.4931 - acc: 0.8153 - val_loss: 2.7305 - val_acc: 0.5000
Epoch 66/100
249/249 [==============================] - 0s 170us/sample - loss: 0.4930 - acc: 0.8594 - val_loss: 2.7435 - val_acc: 0.5000
Epoch 67/100
249/249 [=========================

In [351]:
model.load_weights('best_model.hdf5')

In [352]:
test_predictions = model.predict(Xtest)

# Show Test Results

In [360]:
threshold = .2
ground_truths = []
predictions = []

for i, data in enumerate(zip(mlb.inverse_transform(test_predictions >= threshold), mlb.inverse_transform(ytest))):
    pred, ground_truth = data
    ground_truths.append([ground_truth])
    predictions.append([pred])

In [361]:
pd.DataFrame({'ground_truth':ground_truths, 'prediction': predictions})

,ground_truth,prediction
0,"[(community/question,)]",[()]
1,"[(cla: yes,)]","[(cla: yes,)]"
2,"[(area/build-release,)]",[()]
3,"[(area/kfctl,)]",[()]
4,"[(community/question,)]","[(community/question,)]"
5,"[(area/bootstrap, area/testing, platform/gcp)]",[()]
6,"[(area/bootstrap,)]",[()]
7,"[(cuj/multi-user,)]",[()]
8,"[(cla: yes,)]",[()]
9,"[(cla: yes,)]","[(cla: yes,)]"


## Conclusion:  

This simple model sucks, need to find something more compelling in this situation.